## Overview

## Setup

In [1]:
import os
from tqdm import tqdm
import tensorflow as tf
import glob
import numpy as np
from sklearn.metrics import accuracy_score
import time
import pandas as pd
print(tf.__version__)    

2.6.0


Define models

In [2]:

model_pathes = []
names = []
for file in glob.glob("../new_models/*.tflite"):
    model_pathes.append(file)
    names.append(file[14:])    
    

In [3]:
names

['actually_quantized_model_for_the_TFLite_backend.tflite',
 'aware_quantized_tflite_model.tflite',
 'only_16_bit_activations_with_8_bit_weights.tflite',
 'post_training_dynamic_range_quantization.tflite',
 'post_training_float16_quantization_model.tflite',
 'pruned_tflite.tflite',
 'quantized_and_pruned_tflite.tflite',
 'sparsity_clustered_model_tflite_file.tflite']

In [4]:
def get_ds():
    """
    ## Generate a `Dataset`
    """

    image_size = (180, 180)
    batch_size = 32

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "../PetImages",
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "../PetImages",
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )

    def scale(image, label):
        return tf.image.convert_image_dtype(image, tf.float32)/255, label

    train_ds = train_ds.map(scale)
    val_ds = val_ds.map(scale)
    
    return train_ds, val_ds

In [5]:
def evaluate_model(interpreter):
    
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on ever y image in the "test" dataset.
    prediction = []
    y_true = []

    
    with tf.device('/cpu:0'):

    
        for i, test_image in enumerate(tqdm(test_images)):    

            if i == 500:  # Ограничемся 500-ми первыми (очень долго ждать всех...)          
                break

            # Pre-processing: add batch dimension and convert to float32 to match with
            # the model's input data format.
            test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
            interpreter.set_tensor(input_index, test_image)

            # Run inference.
            interpreter.invoke()

            # Post-processing: remove batch dimension and find the digit with highest
            # probability.
            output = interpreter.tensor(output_index)

            if output()[0][0] > 0.5:
                prediction.append(1)
            else:
                prediction.append(0)            
            y_true.append(test_labels[i])

    print('\n')
    prediction = np.array(prediction)
    y_true = np.array(y_true)    
    accuracy = accuracy_score(y_true, prediction)
    return accuracy

In [6]:
_, val_ds = get_ds()

line = []
test_images = []
test_labels =[]
for element in val_ds.as_numpy_iterator():

    temp = list(element[1].reshape(-1))
    test_labels = test_labels + temp
    a = element[0]
    temp = list(a)  
    test_images = test_images + temp    


test_images = np.array(test_images)
test_labels = np.array(test_labels)    

Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Found 23422 files belonging to 2 classes.
Using 4684 files for validation.


In [7]:
for i in range(len(model_pathes)):

    interpreter = tf.lite.Interpreter(model_path=model_pathes[i])
    interpreter.allocate_tensors()
    
    start_time = time.time()    
    test_accuracy = evaluate_model(interpreter)        
    run_time = time.time() - start_time
    
    size = os.path.getsize(model_pathes[i])
    line.append([names[i], test_accuracy, round(run_time, 3), size])
    print(line[i])
    df = pd.DataFrame(line)
    df.columns = ['name', 'accuracy', 'run time cpu only', 'size']
    df.to_csv('results_no_gpu.csv')


    

  0%|                                                                                                                                                                                                            | 0/4684 [00:00<?, ?it/s]



['actually_quantized_model_for_the_TFLite_backend.tflite', 0.956, 2779.818, 3039968]


  0%|                                                                                                                                                                                                            | 0/4684 [00:00<?, ?it/s]



['aware_quantized_tflite_model.tflite', 0.936, 2782.475, 3041128]


  0%|                                                                                                                                                                                                            | 0/4684 [00:00<?, ?it/s]

...



['pruned_tflite.tflite', 0.954, 16.077, 11084464]


  0%|                                                                                                                                                                                                            | 0/4684 [00:00<?, ?it/s]



['quantized_and_pruned_tflite.tflite', 0.956, 2741.036, 2997200]


 11%|████████████████████▍                                                                                                                                                                           | 500/4684 [45:42<6:22:31,  5.49s/it]



['sparsity_clustered_model_tflite_file.tflite', 0.952, 2742.798, 2997200]


In [8]:
line.append(['BASE MOSEL', 0.952, 30.200, 34768813])

df = pd.DataFrame(line)
df.columns = ['name', 'accuracy', 'run time cpu only', 'size']
df.to_csv('results_no_gpu.csv')

In [9]:
df = df.sort_values(by='size', ascending = True )
df

,name,accuracy,run time cpu only,size
3,post_training_dynamic_range_quantization.tflite,0.956,2742.804,2997200
6,quantized_and_pruned_tflite.tflite,0.956,2741.036,2997200
7,sparsity_clustered_model_tflite_file.tflite,0.952,2742.798,2997200
0,actually_quantized_model_for_the_TFLite_backen...,0.956,2779.818,3039968
1,aware_quantized_tflite_model.tflite,0.936,2782.475,3041128
2,only_16_bit_activations_with_8_bit_weights.tflite,0.954,340.806,3120768
4,post_training_float16_quantization_model.tflite,0.952,16.130,5559600
5,pruned_tflite.tflite,0.954,16.077,11084464
8,BASE MOSEL,0.952,30.200,34768813


In [10]:
df = df.sort_values(by='run time cpu only', ascending = True )
df

,name,accuracy,run time cpu only,size
5,pruned_tflite.tflite,0.954,16.077,11084464
4,post_training_float16_quantization_model.tflite,0.952,16.130,5559600
8,BASE MOSEL,0.952,30.200,34768813
2,only_16_bit_activations_with_8_bit_weights.tflite,0.954,340.806,3120768
6,quantized_and_pruned_tflite.tflite,0.956,2741.036,2997200
7,sparsity_clustered_model_tflite_file.tflite,0.952,2742.798,2997200
3,post_training_dynamic_range_quantization.tflite,0.956,2742.804,2997200
0,actually_quantized_model_for_the_TFLite_backen...,0.956,2779.818,3039968
1,aware_quantized_tflite_model.tflite,0.936,2782.475,3041128


In [11]:
df = df.sort_values(by='accuracy', ascending = False )
df

,name,accuracy,run time cpu only,size
6,quantized_and_pruned_tflite.tflite,0.956,2741.036,2997200
3,post_training_dynamic_range_quantization.tflite,0.956,2742.804,2997200
0,actually_quantized_model_for_the_TFLite_backen...,0.956,2779.818,3039968
5,pruned_tflite.tflite,0.954,16.077,11084464
2,only_16_bit_activations_with_8_bit_weights.tflite,0.954,340.806,3120768
4,post_training_float16_quantization_model.tflite,0.952,16.130,5559600
8,BASE MOSEL,0.952,30.200,34768813
7,sparsity_clustered_model_tflite_file.tflite,0.952,2742.798,2997200
1,aware_quantized_tflite_model.tflite,0.936,2782.475,3041128
